In [ ]:
%pip install requests-cache

### Import Necessary Packages

In [ ]:
import requests_cache
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import os
import datetime

from lts import LTSCell

### Download Data

In [ ]:
#Get data
df=yf.download("GOOGL", period="1y", interval="1d")
data = pd.DataFrame(df)
data.head()

### Normalize the data

__You can get usable signals by using sine and cosine transforms to clear "Time of year" signals__

In [ ]:
data['Date'] = df.index
x = len(data)
data["num_index"] = range(0, x, 1)
data.set_index(df['num_index'], inplace=True)

date_time = pd.to_datetime(data.pop('Date'), format='%d.%m.%Y %H:%M:%S')

timestamp_s = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day

data['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
data['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
#Remove unnecessary columns
data.drop(columns=[
    'Adj Close'],
    inplace=True)

data.head()

In [ ]:
data.info()

In [ ]:
timesteps = data.index.to_numpy()
financials = data.to_numpy()

In [ ]:
timesteps, financials

In [ ]:
# predict 1 step at a time
HORIZON = 1

# use a week worth of timesteps to predict the horizon
WINDOW_SIZE = 7

In [ ]:
# Model checkpoint with a specific filename
def create_model_checkpoint(model_name, save_path='model_checkpoints', monitor_dataset_loss=False):
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(save_path, model_name),
        verbose=0,
        save_best_only=True,
        #monitor='loss' if monitor_dataset_loss else 'val_loss',
        monitor='loss'
    )

# Create a tensorboard callback
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
    )

    print(f'Saving TensorBoard log files to: {log_dir}')
    return tensorboard_callback

# Create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=200,
    restore_best_weights=True
)

# create reduce lr on plateau callback
reduce_lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    patience=100,
    verbose=1
)

In [ ]:
# Create sequential splits at a specified point (80% train, 20% test)
split_size = int(.8 * len(financials))

x_train, y_train = timesteps[:split_size], financials[:split_size]
x_test, y_test = timesteps[split_size:], financials[split_size:]

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
model_lts = tf.keras.Sequential([
    tf.keras.layers.RNN(LTSCell(32), time_major=True, return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(HORIZON, activation='linear')
], name='model_lts')

In [ ]:
model_lts.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = .01),
    loss = tf.keras.losses.MAE,
    metrics = ['mae', 'mse']
)

In [ ]:
history = model_lts.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=500,
    callbacks=[
        create_model_checkpoint(model_name=model_lts.name),
        create_tensorboard_callback(
            dir_name='tensorboard_logs/tensorboard_logs_model_lts',
            experiment_name='model_lts'
        ),
        early_stopping,
        reduce_lr_plateau,
        
    ]
)